In [1]:
# imports
import os
import pickle

import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from matplotlib import gridspec
import sklearn
from sklearn import decomposition, cluster
from sklearn.linear_model import LinearRegression as linreg


import TwoPUtils



%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
with open("/home/mplitt/2P_scratch/all_session_dict.pkl","rb") as f:
    all_sessions = pickle.load(f)

In [3]:
mouse = "GRABDA7"
ind = 2

f = all_sessions[mouse][ind]
f['mouse']=mouse


scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
try:
    source_folder  = os.path.join('/media/mplitt','Backup Plus')
    source_stem = os.path.join(source_folder,f['mouse'],f['date'],f['scene'],scan_str)
    info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
except:
    source_folder  = os.path.join('/media/mplitt','Backup Plus1','2P_Data')
    source_stem = os.path.join(source_folder,f['mouse'],f['date'],f['scene'],scan_str)
    info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')

f.update({'scan_file': source_stem + '.sbx',
          'scanheader_file': source_stem + '.mat',
          'vr_filename': os.path.join("/home/mplitt/VR_scratch",f['mouse'],f['date'],"%s_%d.sqlite" %(f['scene'],f['session'])),
          'scan_number': f['scan'],
          'prompt_for_keys': False,
          'VR_only': False,
          'scanner': "NLW",
         })
if f['mouse'] == 'GRABDA6':
    f['s2p_path']=os.path.join("/home/mplitt/2P_scratch",f['mouse'],f['date'],f['scene'],scan_str,'suite2p')
else:
    f['s2p_path']=os.path.join("/mnt/BigDisk/2P_scratch",f['mouse'],f['date'],f['scene'],scan_str,'suite2p')
    
print(f)


{'date': '12_12_2020', 'scene': 'NeuroMods_LocationA', 'session': 2, 'scan': 2, 'mouse': 'GRABDA7', 'scan_file': '/media/mplitt/Backup Plus/GRABDA7/12_12_2020/NeuroMods_LocationA/NeuroMods_LocationA_002_002.sbx', 'scanheader_file': '/media/mplitt/Backup Plus/GRABDA7/12_12_2020/NeuroMods_LocationA/NeuroMods_LocationA_002_002.mat', 'vr_filename': '/home/mplitt/VR_scratch/GRABDA7/12_12_2020/NeuroMods_LocationA_2.sqlite', 'scan_number': 2, 'prompt_for_keys': False, 'VR_only': False, 'scanner': 'NLW', 's2p_path': '/mnt/BigDisk/2P_scratch/GRABDA7/12_12_2020/NeuroMods_LocationA/NeuroMods_LocationA_002_002/suite2p'}


In [5]:
sess = TwoPUtils.sess.Session(**f)
sess.load_scan_info()
sess.align_VR_to_2P()
sess.load_suite2p_data()
sess.add_timeseries(licks=sess.vr_data['lick'],rewards=sess.vr_data['reward'])
sess.add_pos_binned_trial_matrix(['licks','rewards'],'pos')



/home/mplitt/repos/TwoPUtils/sess.py:93: UserWarning: skipping checking keys, remaining initialization not guaranteed to work
  warnings.warn("skipping checking keys, remaining initialization not guaranteed to work")
/home/mplitt/repos/TwoPUtils/sess.py:256: UserWarning: Looking for coaligned suite2p sessions is not implemented yet
  warnings.warn("Looking for coaligned suite2p sessions is not implemented yet")
/home/mplitt/repos/TwoPUtils/preprocessing.py:134: RuntimeWarning: divide by zero encountered in remainder
  lines = np.array([l % scan_info['fold_lines'] for l in scan_info['line']])


1510.645780697322 1510.5811015664478


/home/mplitt/repos/TwoPUtils/sess.py:390: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  v = v[np.newaxis, :]


In [6]:
# calculate place cells
#regress green channel from red channel

F = np.zeros(sess.timeseries['F'].shape)*np.nan
F2 = np.zeros(sess.timeseries['F'].shape)*np.nan

Fneu = np.zeros(sess.timeseries['F'].shape)*np.nan
Fneu2 = np.zeros(sess.timeseries['F'].shape)*np.nan
for i, (start,stop) in enumerate(zip(sess.trial_start_inds.tolist(),sess.teleport_inds.tolist())):
    F[:,start-1:stop-1] = sess.timeseries['F'][:,start-1:stop-1]
    F2[:,start-1:stop-1] = sess.timeseries['F_chan2'][:,start-1:stop-1]
    
    Fneu[:,start-1:stop-1] = sess.timeseries['Fneu'][:,start-1:stop-1]
    Fneu2[:,start-1:stop-1] = sess.timeseries['Fneu_chan2'][:,start-1:stop-1]
    
nanmask = ~np.isnan(F[0,:])
print(nanmask.sum())
for cell in range(F.shape[0]):
    lr = linreg().fit(F2[cell:cell+1,nanmask].T,F[cell,nanmask])
    F[cell,nanmask] = F[cell,nanmask]-lr.predict(F2[cell:cell+1,nanmask].T) + lr.intercept_
    
    lr = linreg().fit(Fneu2[cell:cell+1,nanmask].T,Fneu[cell,nanmask])
    Fneu[cell,nanmask] = Fneu[cell,nanmask]-lr.predict(Fneu2[cell:cell+1,nanmask].T) + lr.intercept_
    
F -= .7*Fneu
F2 -= .7*Fneu2
    


Flow = sp.ndimage.filters.gaussian_filter(F[:,nanmask],    [0., 15])
Flow = sp.ndimage.filters.minimum_filter1d(Flow,    int(500*15))
Flow = sp.ndimage.filters.maximum_filter1d(Flow,    int(500*15))
dFF = np.zeros(sess.timeseries['F'].shape)*np.nan
dFF[:,nanmask] = (F[:,nanmask]-Flow)/np.abs(Flow)



Flow = sp.ndimage.filters.gaussian_filter(F2[:,nanmask],    [0., 15])
Flow = sp.ndimage.filters.minimum_filter1d(Flow,    int(500*15))
Flow = sp.ndimage.filters.maximum_filter1d(Flow,    int(500*15))
dFF2 = np.zeros(sess.timeseries['F'].shape)*np.nan
dFF2[:,nanmask] = (F2[:,nanmask]-Flow)/np.abs(Flow)


for i, (start,stop) in enumerate(zip(sess.trial_start_inds.tolist(),sess.teleport_inds.tolist())):
    dFF[:,start-1:stop-1] = sp.ndimage.filters.gaussian_filter1d(dFF[:,start-1:stop-1],2,axis=1)
    dFF2[:,start-1:stop-1] = sp.ndimage.filters.gaussian_filter1d(dFF2[:,start-1:stop-1],2,axis=1)

sess.add_timeseries(dff=dFF, dff2=dFF2)
sess.add_pos_binned_trial_matrix(['dff','dff2'],'pos')

# for cell in range(0,dFF.shape[0],50):
#     fig,ax = plt.subplots(figsize=[15,3])
#     ax.plot(dFF[cell,:5000])
#     ax.plot(dFF2[cell,:5000])


18070


In [7]:
# run pca on average dopamine responses




In [8]:
# sort cells by first PC


In [ ]:
# plot ROI masks colored by 